In [2]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.stem import WordNetLemmatizer

In [3]:
df = pd.read_csv("train.csv", encoding='ISO-8859-1')
df.head()


,textID,text,selected_text,sentiment,Time of Tweet,Age of User,Country,Population -2020,Land Area (Km²),Density (P/Km²)
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral,morning,0-20,Afghanistan,38928346,652860.0,60
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative,noon,21-30,Albania,2877797,27400.0,105
2,088c60f138,my boss is bullying me...,bullying me,negative,night,31-45,Algeria,43851044,2381740.0,18
3,9642c003ef,what interview! leave me alone,leave me alone,negative,morning,46-60,Andorra,77265,470.0,164
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative,noon,60-70,Angola,32866272,1246700.0,26


In [4]:
array_1=df.iloc[:,1:4].astype(str).applymap(str.lower)
array_1

,text,selected_text,sentiment
0,"i`d have responded, if i were going","i`d have responded, if i were going",neutral
1,sooo sad i will miss you here in san diego!!!,sooo sad,negative
2,my boss is bullying me...,bullying me,negative
3,what interview! leave me alone,leave me alone,negative
4,"sons of ****, why couldn`t they put them on t...","sons of ****,",negative
...,...,...,...
27476,wish we could come see u on denver husband l...,d lost,negative
27477,i`ve wondered about rake to. the client has ...,", don`t force",negative
27478,yay good for both of you. enjoy the break - y...,yay good for both of you.,positive
27479,but it was worth it ****.,but it was worth it ****.,positive


In [5]:
punctuations = ['!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', '{', '|', '}', '~']
stop_words = [
    'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves',
    'you', 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself',
    'she', 'her', 'hers', 'herself', 'it', 'its', 'itself', 'they', 'them', 'their',
    'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', 'these',
    'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has',
    'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but',
    'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with',
    'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',
    'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over',
    'under', 'again', 'further', 'then', 'once'
]


In [6]:
lemmatizer=WordNetLemmatizer()

In [10]:
columns_to_tokenize=['text','selected_text']
def tokenize_and_get_tokens(text):
    tokens=word_tokenize(text)  # tokenizing each word in the text
    token_numbers=[]
    for i, token in enumerate(tokens,start=1):
        if token not in punctuations and token.lower() not in stop_words:     #removing the special character that do not have any use on sentiment analysis
            lemma=lemmatizer.lemmatize(token.lower())
            token_numbers.append((i,lemma))
    return token_numbers
for col in columns_to_tokenize:
    array_1[col] = array_1[col].apply(tokenize_and_get_tokens)

In [11]:
vectorizer = CountVectorizer(max_features=100)
X_train = vectorizer.fit_transform(array_1['text'].apply(lambda tokens: ' '.join([word for (position, word) in tokens])))
y_train=array_1['sentiment']

In [12]:
print("Feature names:", vectorizer.get_feature_names_out())

Feature names: ['all' 'back' 'bad' 'bed' 'better' 'bit' 'can' 'com' 'come' 'could' 'day'
 'didn' 'don' 'feel' 'friend' 'fun' 'get' 'getting' 'go' 'going' 'gon'
 'good' 'got' 'great' 'haha' 'happy' 'hate' 'here' 'home' 'hope' 'hour'
 'how' 'http' 'im' 'just' 'know' 'last' 'like' 'll' 'lol' 'look' 'love'
 'make' 'miss' 'mom' 'more' 'morning' 'mother' 'much' 'na' 'need' 'new'
 'next' 'nice' 'night' 'no' 'not' 'now' 'oh' 'one' 'only' 'people' 're'
 'really' 'right' 'sad' 'say' 'see' 'should' 'sleep' 'so' 'some' 'sorry'
 'still' 'thanks' 'there' 'thing' 'think' 'though' 'time' 'today'
 'tomorrow' 'tonight' 'too' 'twitpic' 'twitter' 've' 'very' 'want' 'way'
 'week' 'weekend' 'well' 'when' 'why' 'will' 'wish' 'work' 'would' 'yeah']


In [13]:
classifier = MultinomialNB()
classifier.fit(X_train, y_train)

MultinomialNB()

In [19]:
# Preprocess the new text
new_text = "This is a bad movie"
new_text_tokens = tokenize_and_get_tokens(new_text)  # Assuming you have the tokenization function defined

# Transform the new text into features using the existing CountVectorizer
new_text_features = vectorizer.transform([' '.join([word for (position, word) in new_text_tokens])])

# Make a prediction
predicted_sentiment = classifier.predict(new_text_features)[0]

# Interpret the prediction
print("Predicted sentiment:", predicted_sentiment)


Predicted sentiment: negative
